In [1]:
import itertools
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh. io import output_notebook
from bokeh.layouts import column, row
from bokeh.models import Span, HoverTool, Label, ColumnDataSource

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [4]:
combined_df = pd.read_csv('../data/processed/processed.csv')

In [5]:
combined_df.columns

Index(['date', 'total_for_foreign', 'total_cash', 'grand_total',
       'exchange_rate', 'url'],
      dtype='object')

In [6]:
combined_df.head()

,date,total_for_foreign,total_cash,grand_total,exchange_rate,url
0,تـاريخ البيع - الثلاثاء 19-9-2017 والمنفذ يوم ...,118812407.0,2160000.0,162772407.0,سعر بيع المبالغ المحولة لحسابات المصارف في الخ...,https://cbi.iq/currency_auction/view/124
1,تـاريخ البيع - الاربعاء 20-9-2017 والمنفذ يوم ...,128664984.0,810000.0,163974984.0,سعر بيع المبالغ المحولة لحسابات المصارف في الخ...,https://cbi.iq/currency_auction/view/125
2,تـاريخ البيع - الخميس 21-9-2017 والمنفذ يوم ال...,145291507.0,16950000.0,162241507.0,سعر بيع المبالغ المحولة لحسابات المصارف في الخ...,https://cbi.iq/currency_auction/view/126
3,تـاريخ البيع - الاحد 24-9-2017,111966241.0,41260000.0,153226241.0,سعر بيع المبالغ المحولة لحسابات المصارف في الخ...,https://cbi.iq/currency_auction/view/127
4,تـاريخ البيع - الاثنين 25-9-2017 والمنفذ يوم ا...,113115408.0,40020000.0,153135408.0,سعر بيع المبالغ المحولة لحسابات المصارف في الخ...,https://cbi.iq/currency_auction/view/128


In [7]:
combined_df['rolling_grand'] = combined_df.grand_total.rolling(7).mean()

In [8]:
plot1 = figure(
    x_axis_label='Date',
    y_axis_label='Amount',
    plot_width=700, 
    plot_height=125
)

plot1.line(
    combined_df.index,
    combined_df.grand_total,
    line_width=2,
    line_color='red'
) # line glyph acts much like the circle one

plot2 = figure(
    x_axis_label='Date',
    y_axis_label='Amount',
    plot_width=700, 
    plot_height=125
)


plot2.line(
    combined_df.index,
    combined_df.rolling_grand,
    line_width=2,
    line_color='green'
) # line glyph acts much like the circle one


# daylight_savings_end = Span(location=end_date,
#                             dimension='height', line_color='red',
#                             line_dash='dashed', line_width=3)

# first wave line
first_sanct_line = Span(location = 227, dimension='height',line_color='purple')
plot1.add_layout(first_sanct_line)

layout = column([plot1,plot2], sizing_mode='scale_width')

In [9]:
show(layout)

### Re-doing Plotly viz

In [10]:
# get total for foreign and rolling
combined_df['rolling_foreign'] = combined_df.total_for_foreign.rolling(7).mean()
# combined_df['rolling_grand'] = combined_df.grand_total.rolling(7).mean()

In [11]:
# convert to columndatasource
source = ColumnDataSource(combined_df)

In [12]:
source.column_names

['index',
 'date',
 'total_for_foreign',
 'total_cash',
 'grand_total',
 'exchange_rate',
 'url',
 'rolling_grand',
 'rolling_foreign']

In [55]:
plot = figure(
    title='Total for covering foreign accounts since September 2017',
    x_axis_label='Date',
    y_axis_label='Amount',
    plot_width=700, 
    plot_height=425
)

plot.line(
    source=source,
    x="index",
    y="total_for_foreign",
    line_width=2,
    line_color='darkblue',
    alpha=0.5,
    legend_label="Total for foreign"
)

plot.line(
    source=source,
    x="index",
    y="rolling_foreign",
    line_width=2,
    line_color='fuchsia',
    alpha=0.5,
    legend_label="Foreign 7-day rolling avg",
    hover_color='lightpink'
)

# legend
plot.legend.location = "top_left"
plot.legend.click_policy="hide"
plot.legend.background_fill_color = 'white'

# hover = HoverTool(tooltips=[('Amount','$y{int}')])
hover = HoverTool(tooltips=[
    ('total for foreign','@total_for_foreign'),
    ('rolling foreign','@rolling_foreign{int}')
])
plot.add_tools(hover)

# line for sanctions announcement
sanct_announce = Span(
    location = 164,
    dimension='height',
    line_color='darkgray',
    line_dash='dashed'
)
plot.add_layout(sanct_announce)

sanct_announce_label = Label(
    x=164, 
    y=190000000, 
    text='8 May 2018 announcement', 
    text_color='gray',
    border_line_color='gray',
    text_font_size='12px',
    text_align='right'
)
plot.add_layout(sanct_announce_label)

# line for first snapback sanctions
first_snapback = Span(
    location = 227, 
    dimension='height',
    line_color='darkgray',
    line_dash='dashed'
)
plot.add_layout(first_snapback)

first_snapback_label = Label(
    x=227, 
    y=15000000, 
    text='6 Aug 2018 snapback sanctions', 
    text_color='gray',
    border_line_color='gray',
    text_font_size='12px',
    text_align='left'
)
plot.add_layout(first_snapback_label)

second_snapback = Span(
    location = 291, 
    dimension='height',
    line_color='darkgray',
    line_dash='dashed'
)
plot.add_layout(second_snapback)

second_snapback_label = Label(
    x=291, 
    y=39000000, 
    text='4 Nov 2018 snapback sanctions', 
    text_color='gray',
    border_line_color='gray',
    text_font_size='12px',
    text_align='left'
)
plot.add_layout(second_snapback_label)


plot.xaxis.ticker = [76, 334,583]
plot.xaxis.major_label_overrides = {76: '2018-01-01', 334: '2019-01-01',583:'2020-01-01'}

# turn off scientific notation for the y axis
plot.yaxis.formatter.use_scientific = False

layout = column([plot], sizing_mode='scale_width')

In [56]:
show(layout)